Create a schema

In this case we will reuse the one inferred by Spark

In [1]:
df = spark.read.json("/work/data/covid-tweets")
schema = df.schema

In [2]:
df.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



Create the streaming entry point using the previously loaded schema

In [3]:
streaming = spark.readStream\
    .schema(schema)\
    .option("maxFilesPerTrigger", 1)\
    .json("/work/data/covid-tweets")

In [4]:
activityCounts = streaming.groupBy("gt").count()

In [5]:
activityQuery = activityCounts.writeStream.queryName("activity_counts")\
  .format("memory").outputMode("update")\
  .start()

22/11/21 20:59:00 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-06f2acc1-cbb5-4721-8782-6159ca2e4191. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [6]:
spark.streams.active

In [7]:
from time import sleep
from IPython.display import display, clear_output
for x in range(10):
    clear_output(wait=True)
    spark.sql("SELECT * FROM activity_counts limit 10").show()
    sleep(10)

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup| 5187|
|       sit| 7209|
|     stand| 6268|
|      walk| 6991|
|      bike| 4019|
|stairsdown| 4814|
|      null| 4518|
|  stairsup|10452|
|       sit|12309|
|     stand|11384|
+----------+-----+

